# データ解析体験｜GoogleDrive上の Excelデータの活用事例

## データの読み込み

In [1]:
# pandasをインポート
import pandas as pd

# Google Drive 接続準備
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
# StudentsData.xlsx の読み込み
df = pd.read_excel('/content/drive/My Drive/DataScience/StudentsData.xlsx')

df

,ID,学部,性別,成績,出席率,国語,英語,数学,出身県
0,1,芸術,男,1.962,0.90,52.0,21.0,NaN,福岡県
1,2,理工,男,2.224,0.94,68.0,48.0,NaN,山口県
2,3,経済,男,1.355,0.58,64.0,7.0,35.0,長崎県
3,4,経済,女,1.847,0.65,NaN,NaN,NaN,福岡県
4,5,芸術,女,3.200,0.95,64.0,35.0,NaN,福岡県
...,...,...,...,...,...,...,...,...,...
1312,1313,体育,男,1.716,0.84,62.0,12.0,NaN,福岡県
1313,1314,経済,男,1.629,0.88,42.0,16.0,NaN,福岡県
1314,1315,体育,男,1.490,0.78,56.0,NaN,NaN,福岡県
1315,1316,芸術,女,3.080,0.93,68.0,67.0,NaN,宮崎県


In [30]:
# データフレームの基本情報
print('行数・列数：', df.shape)
print('項目名：', df.columns)
print('各列のデータ型：\n', df.dtypes)

行数・列数： (1317, 9)
項目名： Index(['ID', '学部', '性別', '成績', '出席率', '国語', '英語', '数学', '出身県'], dtype='object')
各列のデータ型：
 ID       int64
学部      object
性別      object
成績     float64
出席率    float64
国語     float64
英語     float64
数学     float64
出身県     object
dtype: object


df.dtypes で表示される「データ型」の違いが重要です。
* int64 , float64： **数値項目（量的データ）**で、計算の対象になります。int は整数、float は実数です。
* object：**名義項目（質的データ）**で、データを分類する際のカテゴリーキーワードになります。

In [31]:
# 学部別のデータ件数｜欠損値削除前の全てのデータ件数を確認
df_faculty_counts = df['学部'].value_counts()
df_faculty_counts

経済    442
体育    287
芸術    255
理工    243
文学     90
Name: 学部, dtype: int64

## データの前処理


In [32]:
# ID の列を削除｜単なる通し番号で、処理には不要なため
df.drop( columns='ID', inplace=True )

In [33]:
# 欠損値の確認
df.isnull().sum()

学部       0
性別       0
成績       0
出席率      0
国語      45
英語      49
数学     746
出身県      0
dtype: int64

数学の試験を受けていない学生が多いようです。

In [34]:
# 欠損値を含む行を削除
df = df.dropna(how='any')

# 状態の確認
print(df.shape)
df.isnull().sum()

(569, 8)


学部     0
性別     0
成績     0
出席率    0
国語     0
英語     0
数学     0
出身県    0
dtype: int64

In [35]:
# 学部別のデータ件数｜欠損データ削除後の状態を確認
df_faculty_counts = df['学部'].value_counts()
df_faculty_counts

理工    169
体育    162
経済    132
文学     90
芸術     16
Name: 学部, dtype: int64

数学の点数が存在しない学生のデータを削除したら、芸術学部のデータが極端に減ったことがわかります。

In [36]:
# 要約統計量を表示
df.describe()

,成績,出席率,国語,英語,数学
count,569.000000,569.000000,569.000000,569.000000,569.000000
mean,2.296445,0.852496,57.752197,48.495606,45.775044
std,0.777337,0.120802,12.936196,27.103032,25.037901
min,0.000000,0.300000,12.000000,0.000000,0.000000
25%,1.750000,0.790000,50.000000,26.000000,25.000000
50%,2.346000,0.890000,58.000000,47.000000,40.000000
75%,2.882000,0.940000,68.000000,73.000000,65.000000
max,3.870000,0.990000,90.000000,99.000000,100.000000


In [37]:
# 相関行列を表示
df.corr(numeric_only=True)

,成績,出席率,国語,英語,数学
成績,1.000000,0.747990,0.334020,0.474835,0.315454
出席率,0.747990,1.000000,0.085331,0.221600,0.229780
国語,0.334020,0.085331,1.000000,0.570459,0.378227
英語,0.474835,0.221600,0.570459,1.000000,0.527875
数学,0.315454,0.229780,0.378227,0.527875,1.000000


* 成績（GPA）と出席率の間に高い相関があることがわかります。
* 成績（GPA）と最も相関が高いのは「英語」の点数であることがわかります。

## データの抽出事例

In [38]:
# 理工学部生かつ福岡県出身の行のみ抽出
df_select = df.query( ' 学部 =="理工" and 出身県 == "福岡県" ')
df_select

,学部,性別,成績,出席率,国語,英語,数学,出身県
6,理工,男,2.324,0.98,54.0,56.0,80.0,福岡県
12,理工,女,3.083,0.95,76.0,87.0,80.0,福岡県
15,理工,男,2.000,0.94,52.0,21.0,60.0,福岡県
30,理工,男,1.125,0.84,44.0,15.0,36.0,福岡県
32,理工,男,1.808,0.94,52.0,31.0,24.0,福岡県
...,...,...,...,...,...,...,...,...
1249,理工,男,2.690,0.93,70.0,79.0,80.0,福岡県
1293,理工,男,2.083,0.85,50.0,56.0,76.0,福岡県
1297,理工,女,3.314,0.97,82.0,89.0,76.0,福岡県
1301,理工,男,1.448,0.90,38.0,12.0,56.0,福岡県


In [39]:
# 要約統計量を表示｜前提：理工学部生かつ福岡県出身
df_select.describe()

,成績,出席率,国語,英語,数学
count,106.000000,106.000000,106.000000,106.000000,106.000000
mean,2.191538,0.887264,57.216981,51.839623,60.188679
std,0.761390,0.100034,13.497093,27.181274,28.006160
min,0.000000,0.510000,30.000000,1.000000,4.000000
25%,1.514000,0.840000,48.000000,31.000000,40.000000
50%,2.233000,0.920000,58.000000,56.000000,64.000000
75%,2.830250,0.960000,66.000000,73.000000,84.000000
max,3.870000,0.990000,88.000000,99.000000,100.000000


In [40]:
# 相関係数を表示｜前提：理工学部生かつ福岡県出身
df_select.corr(numeric_only=True)

,成績,出席率,国語,英語,数学
成績,1.000000,0.747135,0.389343,0.472719,0.488327
出席率,0.747135,1.000000,0.097081,0.086667,0.219724
国語,0.389343,0.097081,1.000000,0.560437,0.456830
英語,0.472719,0.086667,0.560437,1.000000,0.614949
数学,0.488327,0.219724,0.456830,0.614949,1.000000


## 様々な統計解析

In [41]:
# 学部別の平均｜全ての数値項目
df_faculty_mean = df.groupby('学部').mean(numeric_only=True)
df_faculty_mean

,成績,出席率,国語,英語,数学
学部,,,,,
体育,2.181858,0.826543,55.962963,42.858025,37.888889
文学,2.701511,0.894333,59.377778,52.300000,41.166667
理工,2.319320,0.896272,58.881657,55.426036,61.230769
経済,2.126152,0.797121,57.560606,44.553030,39.606061
芸術,2.341438,0.874375,56.375000,43.500000,39.187500


In [42]:
# 学部別の平均｜特定項目
df_faculty_mean[ ['成績'] ]

,成績
学部,
体育,2.181858
文学,2.701511
理工,2.319320
経済,2.126152
芸術,2.341438


In [43]:
# 学部別の平均｜特定項目・複数
df_faculty_mean[ ['成績' , '出席率'] ]

,成績,出席率
学部,,
体育,2.181858,0.826543
文学,2.701511,0.894333
理工,2.319320,0.896272
経済,2.126152,0.797121
芸術,2.341438,0.874375


In [44]:
# 学部 X 性別 ごとの平均
df_faculty2_mean = df.groupby( [ '学部' ,'性別' ] ).mean(numeric_only=True)
df_faculty2_mean

成績       出席率         国語         英語         数学
学部 性別                                                     
体育 女   2.651844  0.852889  61.155556  54.466667  37.488889
   男   2.001094  0.816410  53.965812  38.393162  38.042735
文学 女   2.968750  0.917778  63.944444  61.277778  41.527778
   男   2.523352  0.878704  56.333333  46.314815  40.925926
理工 女   2.685482  0.931964  63.160714  68.196429  69.357143
   男   2.137858  0.878584  56.761062  49.097345  57.203540
経済 女   2.696417  0.873333  61.916667  61.416667  42.666667
   男   2.069125  0.789500  57.125000  42.866667  39.300000
芸術 女   2.401200  0.868000  62.800000  31.400000  27.800000
   男   2.314273  0.877273  53.454545  49.000000  44.363636

In [45]:
 # クロス集計｜学部別の男女数
cross_table = pd.crosstab( df['学部'] , df['性別'] )
cross_table

性別,女,男
学部,,
体育,45,117
文学,36,54
理工,56,113
経済,12,120
芸術,5,11


In [46]:
# ピボットテーブルを使ってカテゴリごとの統計量を表示
pivot_table = pd.pivot_table(df, index='学部', columns='性別' , values='成績' , aggfunc='mean' )
pivot_table

性別,女,男
学部,,
体育,2.651844,2.001094
文学,2.968750,2.523352
理工,2.685482,2.137858
経済,2.696417,2.069125
芸術,2.401200,2.314273
